In [26]:
import pandas as pd
import numpy as np

title_akas = pd.read_csv('../downloads/title.akas.tsv', delimiter='\t', dtype={'isOriginalTitle': object})
title_basics = pd.read_csv('../downloads/title.basics.tsv', delimiter='\t', dtype={'isAdult': object})
title_crew = pd.read_csv('../downloads/title.crew.tsv', delimiter='\t', dtype={'directors': str})
title_ratings = pd.read_csv('../downloads/title.ratings.tsv', delimiter='\t')
name_basics = pd.read_csv('../downloads/name.basics.tsv', delimiter='\t', dtype={'primaryName': str})

In [27]:
# filter by titleType

title_basics = title_basics[(title_basics['titleType'] == 'movie') | (title_basics['titleType'] == 'tvMovie')]
title_basics.reset_index(inplace=True, drop=True)

# filter by isOriginalTitle

title_akas = title_akas[(title_akas['isOriginalTitle'] == '1') | (title_akas['isOriginalTitle'] == '\\N')]
title_akas.reset_index(inplace=True, drop=True)


In [28]:
# remove duplicate values with null value in 'isOriginalTitle' column

duplicates = title_akas[title_akas.duplicated(['titleId'])]
duplicates = duplicates[(duplicates['isOriginalTitle'] == '\\N') | (duplicates['attributes'] != '\\N')]

title_akas.drop(labels=duplicates.index, axis=0, inplace=True)
title_akas.reset_index(inplace=True, drop=True)

In [29]:
duplicates = title_akas[title_akas.duplicated(['titleId'])]

title_akas.drop(labels=duplicates.index, axis=0, inplace=True)
title_akas.reset_index(inplace=True, drop=True)

In [30]:
# remove movies before a certain year

title_basics['startYear'] = pd.to_numeric(title_basics['startYear'], errors='coerce')
to_remove = title_basics[(title_basics['startYear'] <= 2000) | (title_basics['startYear'] >= 2020)].index
title_basics.drop(to_remove, inplace=True)
title_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
5534,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019.0,\N,\N,"Action,Crime"
24415,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,\N,118,"Comedy,Fantasy,Romance"
45084,tt0061366,movie,Around the World,Around the World,0,NaN,\N,178,"Comedy,Romance"
50120,tt0067683,movie,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006.0,\N,47,Documentary
51206,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,\N,122,Drama
...,...,...,...,...,...,...,...,...,...
760680,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007.0,\N,100,Documentary
760681,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015.0,\N,66,Drama
760682,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013.0,\N,\N,Comedy
760683,tt9916730,movie,6 Gunn,6 Gunn,0,2017.0,\N,116,\N


In [31]:
# drop useless columns 

title_akas.drop(labels=['ordering', 'types', 'attributes', 'title', 'isOriginalTitle'], axis=1, inplace=True)
title_basics.drop(labels=['endYear', 'titleType'], axis=1, inplace=True)
title_crew.drop(labels=['writers'], axis=1, inplace=True)
title_ratings.drop(labels=['numVotes'], axis=1, inplace=True)
name_basics.drop(labels=['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], axis=1, inplace=True)

In [32]:
title_akas['language'].value_counts()

\N     1790881
en           4
tr           1
yue          1
da           1
ca           1
Name: language, dtype: int64

:warning:  language column values have little to none importance, hence not using this dataframe

In [33]:
df = title_basics.merge(
  title_crew, on='tconst', how='left').merge(
    title_ratings, on='tconst', how='left')

df

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,directors,averageRating
0,tt0011801,Tötet nicht mehr,Tötet nicht mehr,0,2019.0,\N,"Action,Crime",nm0681726,NaN
1,tt0035423,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance",nm0003506,6.4
2,tt0061366,Around the World,Around the World,0,NaN,178,"Comedy,Romance",nm1487785,NaN
3,tt0067683,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006.0,47,Documentary,"nm0001425,nm0959099",6.3
4,tt0069049,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,Drama,nm0000080,6.7
...,...,...,...,...,...,...,...,...,...
379759,tt9916680,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007.0,100,Documentary,nm0652213,NaN
379760,tt9916692,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015.0,66,Drama,nm10538592,NaN
379761,tt9916706,Dankyavar Danka,Dankyavar Danka,0,2013.0,\N,Comedy,nm7764440,NaN
379762,tt9916730,6 Gunn,6 Gunn,0,2017.0,116,\N,nm10538612,8.4


In [34]:
# explode movie rows with more than one director into multiple rows, each one with only one director code

df = df.assign(directors=df['directors'].str.split(',')).explode('directors')
df = df.merge(name_basics, left_on='directors', right_on='nconst', how='left')

# drop useless columns

df.drop(labels=['directors', 'nconst'], axis=1, inplace=True)
df.rename(columns={'primaryName' : 'directors'}, inplace=True)

df

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,directors
0,tt0011801,Tötet nicht mehr,Tötet nicht mehr,0,2019.0,\N,"Action,Crime",NaN,Lupu Pick
1,tt0035423,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance",6.4,James Mangold
2,tt0061366,Around the World,Around the World,0,NaN,178,"Comedy,Romance",NaN,Pachhi
3,tt0067683,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006.0,47,Documentary,6.3,Krzysztof Kieslowski
4,tt0067683,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006.0,47,Documentary,6.3,Tomasz Zygadlo
...,...,...,...,...,...,...,...,...,...
426923,tt9916692,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015.0,66,Drama,NaN,Andrzej Bartnikowski
426924,tt9916706,Dankyavar Danka,Dankyavar Danka,0,2013.0,\N,Comedy,NaN,Kanchan Nayak
426925,tt9916730,6 Gunn,6 Gunn,0,2017.0,116,\N,8.4,Kiran Gawade
426926,tt9916754,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,0,2013.0,49,Documentary,NaN,Angela Gurgel


In [35]:
# remove duplicate rows while joining director names

df['directors'] = df['directors'].astype(str)
directors_column = df.groupby(['tconst']).agg({'directors': ', '.join})['directors'].values

df.drop_duplicates(subset='tconst', inplace=True)
df['directors'] = directors_column

df

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,directors
0,tt0011801,Tötet nicht mehr,Tötet nicht mehr,0,2019.0,\N,"Action,Crime",NaN,Lupu Pick
1,tt0035423,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance",6.4,James Mangold
2,tt0061366,Around the World,Around the World,0,NaN,178,"Comedy,Romance",NaN,Pachhi
3,tt0067683,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006.0,47,Documentary,6.3,"Krzysztof Kieslowski, Tomasz Zygadlo"
5,tt0069049,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,Drama,6.7,Orson Welles
...,...,...,...,...,...,...,...,...,...
426922,tt9916680,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007.0,100,Documentary,NaN,Luis Ospina
426923,tt9916692,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015.0,66,Drama,NaN,Andrzej Bartnikowski
426924,tt9916706,Dankyavar Danka,Dankyavar Danka,0,2013.0,\N,Comedy,NaN,Kanchan Nayak
426925,tt9916730,6 Gunn,6 Gunn,0,2017.0,116,\N,8.4,Kiran Gawade


In [36]:
df.to_csv('../data/data.csv', index=False, sep=';')

# TODO:
- remover filmes mais antigos que x data
- filtrar filmes apenas em en 
  - not enough values
- remover filmes sem sinopse (isto é quando formos fazer o webscrapping ig)
